In [1]:
def execute_program(registers, program):
    A, B, C = registers
    output = []
    instruction_pointer = 0

    while instruction_pointer < len(program):
        opcode = program[instruction_pointer]
        operand = program[instruction_pointer + 1]

        def get_combo_value(op):
            if op in range(4):
                return op
            elif op == 4:
                return A
            elif op == 5:
                return B
            elif op == 6:
                return C
            else:
                raise ValueError("Invalid combo operand: {}".format(op))

        if opcode == 0:  # adv
            denom = 2 ** get_combo_value(operand)
            A //= denom
        elif opcode == 1:  # bxl
            B ^= operand
        elif opcode == 2:  # bst
            B = get_combo_value(operand) % 8
        elif opcode == 3:  # jnz
            if A != 0:
                instruction_pointer = operand
                continue
        elif opcode == 4:  # bxc
            B ^= C
        elif opcode == 5:  # out
            output.append(get_combo_value(operand) % 8)
        elif opcode == 6:  # bdv
            denom = 2 ** get_combo_value(operand)
            B = A // denom
        elif opcode == 7:  # cdv
            denom = 2 ** get_combo_value(operand)
            C = A // denom
        else:
            raise ValueError("Invalid opcode: {}".format(opcode))

        instruction_pointer += 2

    return ','.join(map(str, output))

# Initial register values
registers = [32916674, 0, 0]

# Program input
program = [2,4,1,1,7,5,0,3,1,4,4,0,5,5,3,0]

# Execute the program and get the output
output = execute_program(registers, program)
print(output)

7,1,2,3,2,6,7,2,5


In [5]:
def find_quine_value(program):
    def find(target, ans):
        if target == []:  # We've matched all numbers in the target
            return ans
            
        # Try all possible 3-bit values
        for t in range(8):
            a = ans << 3 | t  # Build up the answer by shifting and combining
            b = 0
            c = 0
            output = None
            adv3 = False
            
            def combo(operand):
                if 0 <= operand <= 3: return operand
                if operand == 4: return a
                if operand == 5: return b
                if operand == 6: return c
                raise AssertionError(f"unrecognized combo operand {operand}")
            
            # Execute program until we get an output
            for pointer in range(0, len(program) - 2, 2):
                ins = program[pointer]
                operand = program[pointer + 1]
                
                if ins == 0:  # ADV
                    assert not adv3, "program has multiple ADVs"
                    assert operand == 3, "program has ADV with operand other than 3"
                    adv3 = True
                elif ins == 1:  # XOR immediate
                    b = b ^ operand
                elif ins == 2:  # MOD
                    b = combo(operand) % 8
                elif ins == 3:  # JNZ
                    raise AssertionError("program has JNZ inside expected loop body")
                elif ins == 4:  # XOR C
                    b = b ^ c
                elif ins == 5:  # OUT
                    assert output is None, "program has multiple OUT"
                    output = combo(operand) % 8
                elif ins == 6:  # RSH A
                    b = a >> combo(operand)
                elif ins == 7:  # SET C
                    c = a >> combo(operand)
                
                if output == target[-1]:  # If we found a matching output
                    sub = find(target[:-1], a)  # Recursively try to match the rest
                    if sub is not None:
                        return sub
                    
            # If we get here, this value of t didn't work
            continue
        
        # If we get here, no value of t worked
        return None

    # Start the search with the program as the target
    result = find(program, 0)
    return result

# Your program input
program = [2,4,1,1,7,5,0,3,1,4,4,0,5,5,3,0]

# Find the lowest value of A that makes the program output itself
result = find_quine_value(program)
print(f"Lowest value for register A: {result}")

# Verify the result by running the program with this value
def verify_result(program, a):
    outputs = []
    b = 0
    c = 0
    
    def combo(operand):
        if 0 <= operand <= 3: return operand
        if operand == 4: return a
        if operand == 5: return b
        if operand == 6: return c
        raise AssertionError(f"unrecognized combo operand {operand}")
    
    for pointer in range(0, len(program) - 2, 2):
        ins = program[pointer]
        operand = program[pointer + 1]
        
        if ins == 0: continue  # ADV
        elif ins == 1: b = b ^ operand
        elif ins == 2: b = combo(operand) % 8
        elif ins == 4: b = b ^ c
        elif ins == 5: outputs.append(combo(operand) % 8)
        elif ins == 6: b = a >> combo(operand)
        elif ins == 7: c = a >> combo(operand)
    
    return outputs

if result:
    outputs = verify_result(program, result)

Lowest value for register A: 202356708354602

Verification:

Original program: [2, 4, 1, 1, 7, 5, 0, 3, 1, 4, 4, 0, 5, 5, 3, 0]

Program outputs: [2]

Match: False